In [ ]:
import pandas as pd

# Read Files

In [ ]:
property_df = pd.read_csv("../../data/curated/properties_data4.csv")

# Liveability Index

In [ ]:
# Step 1: Affordability

# Normalizing affordability 
property_df['monthly_income'] = property_df['Median total income (excl. Government pensions and allowances) ($)'] / 12
property_df['rent_income_ratio'] = (property_df['Median weekly household rental payment ($)'] * 4) / property_df['monthly_income']
property_df['normalized_affordability'] = 1 - (property_df['rent_income_ratio'] - property_df['rent_income_ratio'].min())/(
    property_df['rent_income_ratio'].max() - property_df['rent_income_ratio'].min())

# Normalize employment-related columns 
property_df['normalized_employment'] = (property_df["Working age population (aged 15-64 years) (%)"] - 
                                               property_df["Working age population (aged 15-64 years) (%)"].min()) / (
                                                   property_df["Working age population (aged 15-64 years) (%)"].max()
                                                     -property_df["Working age population (aged 15-64 years) (%)"].min())

property_df['normalized_jobs'] = (property_df['Number of jobs'] - property_df['Number of jobs'].min()) / (
    property_df['Number of jobs'].max() - property_df['Number of jobs'].min())

# Combine affordability with employment into the final affordability score
property_df['affordability_score'] = (property_df['normalized_affordability'] +
                                             property_df['normalized_employment'] +
                                             property_df['normalized_jobs']) / 3


In [ ]:
# Step 2: Amenities
property_df['normalized_parks'] = (property_df['park_count_within_1000m'] - property_df['park_count_within_1000m'].min()) / (
    property_df['park_count_within_1000m'].max() - property_df['park_count_within_1000m'].min())

school_count = property_df['kindergarten_count_within_1000m'] + property_df['secondary_primary_school_count_within_3000m']
property_df['normalized_schools'] = (school_count - school_count.min()) / (
    school_count.max() - school_count.min())

property_df['normalized_health'] = 1 - (property_df['distance_to_hospital'] - property_df['distance_to_hospital'].min()) / (
    property_df['distance_to_hospital'].max() - property_df['distance_to_hospital'].min())

property_df['normalized_shops'] = (property_df['shop_count_within_1000m'] - property_df['shop_count_within_1000m'].min()) / (
    property_df['shop_count_within_1000m'].max() - property_df['shop_count_within_1000m'].min())

property_df['amenities_score'] = (property_df['normalized_parks'] +
                           property_df['normalized_schools'] +
                           property_df['normalized_health'] +
                           property_df['normalized_shops']) / 4


In [ ]:
# Step 3: Safety
property_df['normalized_crime'] = 1 - (property_df['avg_crime_count'] - property_df['avg_crime_count'].min()) / (
    property_df['avg_crime_count'].max() - property_df['avg_crime_count'].min())

property_df['safety_score']  = property_df['normalized_crime']

In [ ]:
# Step 4: Accessibility
property_df['normalized_distance_to_cbd'] = 1 - (property_df['distance_to_melbourne_central'] - property_df['distance_to_melbourne_central'].min()) / (
    property_df['distance_to_melbourne_central'].max() - property_df['distance_to_melbourne_central'].min())

property_df['normalized_transport_access'] = 1 - (property_df['route_distance_m'] - property_df['route_distance_m'].min()) / (
    property_df['route_distance_m'].max() - property_df['route_distance_m'].min())

property_df['accessibility_score'] = (property_df['normalized_transport_access'] + property_df['normalized_distance_to_cbd']) / 2


In [ ]:
# Step 5: Calculate the overall Liveability Index as the weighted sum of all factors

weights = {
    'amenities_score': 1,
    'safety_score': 1,
    'accessibility_score': 1,
    'affordability_score': 1
}

property_df['liveability_index'] = (
    property_df['amenities_score'] * weights['amenities_score'] +
    property_df['safety_score'] * weights['safety_score'] +
    property_df['accessibility_score'] * weights['accessibility_score'] +
    property_df['affordability_score'] * weights['affordability_score']
) / sum(weights.values())

top_liveable_property = property_df[['name', 'liveability_index']].sort_values(by='liveability_index', ascending=False).head(10)


In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
top_liveable_property